In [3]:
dt = joblib.load("/Applications/THESIS.joblib")

In [4]:
import pandas as pd
import numpy as np
import joblib

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as lda
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [5]:
filtered_df = dt[ dt.label.isin(["A", "C"]) ] 
filtered_df["numeric_label"] = filtered_df.label.apply(lambda x: 0 if x=='A' else 1 if x=='C' else None)

/var/folders/2r/ng5l8yc97950rj3w9g5mq9h00000gn/T/ipykernel_14368/1446778592.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["numeric_label"] = filtered_df.label.apply(lambda x: 0 if x=='A' else 1 if x=='C' else None)


In [6]:
filtered_df = filtered_df.sample(frac=1)

In [15]:
filtered_df.head(2)

,Fp1_Delta_rel,Fp1_Theta_rel,Fp1_Alpha_rel,Fp1_Sigma_rel,Fp1_Beta_rel,Fp1_Gamma_rel,Fp2_Delta_rel,Fp2_Theta_rel,Fp2_Alpha_rel,Fp2_Sigma_rel,...,Cz_Gamma_rel,Pz_Delta_rel,Pz_Theta_rel,Pz_Alpha_rel,Pz_Sigma_rel,Pz_Beta_rel,Pz_Gamma_rel,group,label,numeric_label
16324,0.8111432963292834,0.10102170490832162,0.05738322754981963,0.006249743216333619,0.017778216091654354,0.006423811904587865,0.7667041997069062,0.1295894631803582,0.06057132835705215,0.009207467850438928,...,0.0027796006896347513,0.8519241627848544,0.07169821281663315,0.05733703285483038,0.006786858435248668,0.009032183029802764,0.0032215500786306715,sub-061,C,1
7338,0.8748613670064123,0.05093324883204172,0.02480754988618224,0.020119180453211708,0.025109360055419423,0.004169293766732759,0.8887790476282915,0.05028711791182438,0.018864641495070566,0.013544380720075447,...,0.0019417543170794373,0.8792164480048873,0.08369071441349829,0.013495385043217366,0.012003055711510346,0.00948093566336631,0.0021134611635203536,sub-018,A,0


In [8]:
features = filtered_df.iloc[:, :-3].to_numpy()
labels = filtered_df.numeric_label.to_numpy()
groups = filtered_df.group.to_numpy()

In [9]:
features.shape, labels.shape, groups.shape

((26526, 114), (26526,), (26526,))

In [10]:
%%time

loso = LeaveOneGroupOut()
feature_selection = True
use_feature_scaler = False
lasso_cv = LassoCV()

result = {
    "tr_label": [],
    "ts_label": [],
    
    "tr_predict": [],
    "ts_predict": [],
    
    "loso_tr_label": [],
    "loso_ts_label": [],
    
    "loso_tr_predict": [],
    "loso_ts_predict": [],
    
    "loso_tr_accuracy": [],
    "loso_ts_accuracy": [],
    
    "loso_tr_f1": [],
    "loso_ts_f1": [],
    
    "loso_tr_sensitivity": [],
    "loso_ts_sensitivity": [],
    
    "loso_tr_specificity": [],
    "loso_ts_specificity": []
}


for i, (tr_idx, ts_idx) in enumerate(loso.split(X=features, y=labels, groups=groups)):

    print(f"""### {i} subject compelete.
        test group: {np.unique(groups[ts_idx])}        
    """)
    
    clf = RandomForestClassifier()
    # clf = SVC()
    # clf = lda()
    
    tr_data, tr_label = features[tr_idx], labels[tr_idx]
    ts_data, ts_label = features[ts_idx], labels[ts_idx]

    if use_feature_scaler:
        scaler = StandardScaler()
        tr_data = scaler.fit_transform(tr_data)
        ts_data = scaler.transform(ts_data)
    
    if feature_selection:
        sfm = SelectFromModel(lasso_cv)
        tr_data = sfm.fit_transform(tr_data, tr_label)
        ts_data = sfm.transform(ts_data)

    clf.fit(tr_data, tr_label)

    result["tr_label"].extend(tr_label.tolist())
    result["ts_label"].extend(ts_label.tolist())
    
    result["tr_predict"].extend(clf.predict(tr_data).tolist())
    result["ts_predict"].extend(clf.predict(ts_data).tolist())
    
    result["loso_tr_label"].append(tr_label)
    result["loso_ts_label"].append(ts_label)
    
    result["loso_tr_predict"].append(clf.predict(tr_data))
    result["loso_ts_predict"].append(clf.predict(ts_data))
    
    result["loso_tr_accuracy"].append(accuracy_score(tr_label, clf.predict(tr_data)))
    result["loso_ts_accuracy"].append(accuracy_score(ts_label, clf.predict(ts_data)))

    result["loso_tr_f1"].append(f1_score(tr_label, clf.predict(tr_data)))
    result["loso_ts_f1"].append(f1_score(ts_label, clf.predict(ts_data)))
    
    result["loso_tr_sensitivity"].append(recall_score(tr_label, clf.predict(tr_data)))
    result["loso_ts_sensitivity"].append(recall_score(ts_label, clf.predict(ts_data)))
    
    result["loso_tr_specificity"].append(precision_score(tr_label, clf.predict(tr_data)))
    result["loso_ts_specificity"].append(precision_score(ts_label, clf.predict(ts_data)))

    print(f"""
            ts acc: {result["loso_ts_accuracy"][-1]}
            ts f1: {result["loso_ts_f1"][-1]} 
            ts sensitivity: {result["loso_ts_sensitivity"][-1]} 
            ts specificity: {result["loso_ts_specificity"][-1]}
    """)
    
    # break

### 0 subject compelete.
        test group: ['sub-001']        
    


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



            ts acc: 0.9630872483221476
            ts f1: 0.0 
            ts sensitivity: 0.0 
            ts specificity: 0.0
    
### 1 subject compelete.
        test group: ['sub-002']        
    


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



            ts acc: 0.46329113924050636
            ts f1: 0.0 
            ts sensitivity: 0.0 
            ts specificity: 0.0
    
### 2 subject compelete.
        test group: ['sub-003']        
    


...


            ts acc: 0.5253807106598984
            ts f1: 0.6888519134775374 
            ts sensitivity: 0.5253807106598984 
            ts specificity: 1.0
    
### 56 subject compelete.
        test group: ['sub-057']        
    

            ts acc: 0.8337531486146096
            ts f1: 0.9093406593406593 
            ts sensitivity: 0.8337531486146096 
            ts specificity: 1.0
    
### 57 subject compelete.
        test group: ['sub-058']        
    

            ts acc: 0.8779840848806366
            ts f1: 0.9350282485875706 
            ts sensitivity: 0.8779840848806366 
            ts specificity: 1.0
    
### 58 subject compelete.
        test group: ['sub-059']        
    

            ts acc: 0.14030612244897958
            ts f1: 0.2460850111856823 
            ts sensitivity: 0.14030612244897958 
            ts specificity: 1.0
    
### 59 subject compelete.
        test group: ['sub-060']        
    

            ts acc: 0.5201072386058981
            ts f1:

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.090e+01, tolerance: 6.436e-01
  model = cd_fast.enet_coordinate_descent(



            ts acc: 0.7834821428571429
            ts f1: 0.8785982478097623 
            ts sensitivity: 0.7834821428571429 
            ts specificity: 1.0
    
### 62 subject compelete.
        test group: ['sub-063']        
    

            ts acc: 0.48129675810473815
            ts f1: 0.6498316498316499 
            ts sensitivity: 0.48129675810473815 
            ts specificity: 1.0
    
### 63 subject compelete.
        test group: ['sub-064']        
    


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.244e-01, tolerance: 6.444e-01
  model = cd_fast.enet_coordinate_descent(



            ts acc: 0.9787234042553191
            ts f1: 0.989247311827957 
            ts sensitivity: 0.9787234042553191 
            ts specificity: 1.0
    
### 64 subject compelete.
        test group: ['sub-065']        
    

            ts acc: 0.528344671201814
            ts f1: 0.6913946587537092 
            ts sensitivity: 0.528344671201814 
            ts specificity: 1.0
    
CPU times: user 1h 35min 30s, sys: 6min 7s, total: 1h 41min 37s
Wall time: 44min 45s


In [11]:
print(f"""
clf: {clf}
feature selection: {feature_selection}
feature selection model: {lasso_cv}
feature scaler: {use_feature_scaler}
ts acc average on loso: {np.mean(result["loso_ts_accuracy"])}
ts acc: {accuracy_score(result["ts_label"], result["ts_predict"])} 
ts f1: {f1_score(result["ts_label"], result["ts_predict"])} 
ts sensitivity: {recall_score(result["ts_label"], result["ts_predict"])} 
ts specificity: {precision_score(result["ts_label"], result["ts_predict"])}
""") 


clf: RandomForestClassifier()
feature selection: True
feature selection model: LassoCV()
feature scaler: False
ts acc average on loso: 0.7174842171629724
ts acc: 0.7192565784513307 
ts f1: 0.6792160241223347 
ts sensitivity: 0.6563436563436563 
ts specificity: 0.7037400696242078



In [12]:
clf.predict(tr_data).shape

(26085,)

In [13]:
recall_score(ts_label, clf.predict(ts_data))

0.528344671201814

In [ ]:
ts_label